In [ ]:
import torch
from mlp import make_dataset, CharacterLevelMLP, Linear, BatchNormalization1D, Tanh
import matplotlib.pyplot as plt
from torch.optim import Adam
from pathlib import Path
from torch.nn.functional import cross_entropy
from sklearn.model_selection import train_test_split
from torch import Tensor
%matplotlib inline

In [2]:
names_file = Path("./names.txt")
context_window = 3
X , Y, itos = make_dataset(names_file, context_window=context_window)

In [3]:
train_X, eval_X, train_Y, eval_Y = train_test_split(X,Y,test_size=0.2)
val_X, test_X, val_Y, test_Y = train_test_split(eval_X,eval_Y,test_size=0.5)

In [ ]:
def cmp(s, dt, t:Tensor):
    exact_match = torch.all(dt == t.grad).item()
    approximate_match = torch.allclose(dt, t.grad)
    max_difference = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact {str(exact_match):5s} | approximate {str(approximate_match):5s} | max_diff {str(max_difference)}")
    